# Vector Search and Prompt Engineering

In the previous Chapter, we created a new database of car manuals with vectorized text.

In this Chapter, we will search for issues which are close to the input query.

## Set up

In [2]:
import cohere
from cassandra import ConsistencyLevel
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster

# get free Trial API Key at https://cohere.ai/
from cred import API_key

co = cohere.Client(API_key)

In [3]:
from cred import (ASTRA_CLIENT_ID, ASTRA_CLIENT_SECRET,
                  SECURE_CONNECT_BUNDLE_PATH)

KEYSPACE_NAME = "demo"
TABLE_NAME = "car_manual_vectorized"

cloud_config = {"secure_connect_bundle": SECURE_CONNECT_BUNDLE_PATH}
auth_provider = PlainTextAuthProvider(ASTRA_CLIENT_ID, ASTRA_CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider, protocol_version=4)
session = cluster.connect()
session.execute(f"USE {KEYSPACE_NAME};")

## Main functions

`answer`: using Generative AI to answer the input question.

`prompt_engineer`: to engineer the input question by adding details of context and narrowing down the copes of the question.

In [30]:
def answer(prompt):
    # The Generative Model to generate answers for
    response = co.generate(  
        model='command', #
        #model= 'command-nightly',   
        prompt = prompt,  
        max_tokens=126,  
        temperature=0.50)
    res = response.generations[0].text

    return res

def prompt_engineer(text, limit=2):
    response = co.embed(model="embed-english-v2.0", texts=[text])
    vec = response.embeddings[0]
    #data = []
    # Prompt Engineering
    prompt = f'Using the following information to answer the question: "{text}"'
    for row in session.execute(
        f"SELECT id, link, html, text, brand, model, year FROM {KEYSPACE_NAME}.{TABLE_NAME} "
        f" ORDER BY html_car_vector_4096 ANN OF %s LIMIT {limit}"
        , [vec]
):
        #data.append([row.id, row.link, row.html, row.text])
        prompt += "\n\n"+row.text
    
    return prompt.strip()

#### Using Generative AI without Prompt Engineering

We can see how AI generate very general answer. It looks like the AI thinks that we are talking about the internet in general.

In [31]:
prompt = "wifi does not work"
print(answer(prompt))


There are a few things you can try to fix a broken wifi connection. First, make sure that your router is properly connected to the internet. If it is, try restarting your router. If that doesn't work, try restarting your computer. If none of those work, you may need to contact your internet service provider for help.


#### Using Generative AI with Prompt Engineering

However, when we told AI about the specific information, it can generate much more informative and relevant answers.

In [33]:
print(answer(prompt_engineer(prompt)))


The answer is Tesla updates your vehicle's software wirelessly, constantly providing new features. Tesla recommends you install software updates at the earliest opportunity on your vehicle. To ensure the fastest and most reliable delivery of software updates, leave Wi-Fi turned on and connected whenever possible. In most cases, your vehicle must be connected to Wi-Fi to start an update (see Wi-Fi).


In [ ]:
# Close connection to Cassandra
cluster.shutdown()